In [7]:
import pandas as pd
from datetime import datetime
import numpy as np

In [13]:
df = pd.read_csv("/home/xlajoie/Downloads/MCI-AD data.csv")

In [15]:
import pandas as pd
import numpy as np

# Your DataFrame is assumed to be already loaded as `df`
# Only keep relevant columns
df = df[['RID', 'Diagnosis', 'VISCODE2']].copy()

# Remove rows with missing VISCODE or Diagnosis
df = df[df['VISCODE2'].notna() & df['Diagnosis'].notna()]

# Convert VISCODE to number of months
def viscode_to_months(viscode):
    if viscode == 'bl':
        return 0
    elif viscode.startswith('m') and viscode[1:].isdigit():
        return int(viscode[1:])
    else:
        return np.nan

df['months'] = df['VISCODE2'].apply(viscode_to_months)

# Group by subject
conversion_months = []
followup_months = []

for rid, group in df.groupby('RID'):
    group = group.sort_values('months')

    # Get baseline month (should always be 0 if 'bl' is present)
    baseline_month = group['months'].min()

    # Time to conversion (first Diagnosis == 3)
    conversion = group[group['Diagnosis'] == 3]
    if not conversion.empty:
        first_conversion_month = conversion['months'].iloc[0]
        months_to_convert = first_conversion_month - baseline_month
        conversion_months.append(months_to_convert)

    # Total follow-up (last visit - baseline)
    last_month = group['months'].max()
    total_followup = last_month - baseline_month
    followup_months.append(total_followup)

# Summary statistics
print(f"✅ Average time to AD conversion (in months): {np.mean(conversion_months):.2f}")
print(f"✅ Average total follow-up time (in months): {np.mean(followup_months):.2f}")



✅ Average time to AD conversion (in months): 31.68
✅ Average total follow-up time (in months): 53.47
